In [1]:
# import the necessary packages
#from keras.applications import ResNet50
#from tensorflow.keras.applications.resnet50 import preprocess_input
#from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from keras.preprocessing.image import img_to_array
from keras.models import load_model
from imutils.object_detection import non_max_suppression
import imutils
import numpy as np
import argparse
import time
import cv2
from PIL import Image

In [2]:
def sliding_window(image, step, ws):
    # slide a window across the image
    for y in range(0, image.shape[0] - ws[1], step):
        for x in range(0, image.shape[1] - ws[0], step):
            # yield the current window
            yield (x, y, image[y:y + ws[1], x:x + ws[0]])

In [3]:
def image_pyramid(image, scale=1.5, minSize=(224, 224)):
    # yield the original image
    yield image
    # keep looping over the image pyramid
    while True:
        # compute the dimensions of the next image in the pyramid
        w = int(image.shape[1] / scale)
        image = imutils.resize(image, width=w)
        # if the resized image does not meet the supplied minimum
        # size, then stop constructing the pyramid
        if image.shape[0] < minSize[1] or image.shape[1] < minSize[0]:
            break
        # yield the next image in the pyramid
        yield image

In [4]:
# initialize variables used for the object detection procedure
WIDTH = 300
PYR_SCALE = 1.5
WIN_STEP = 16
ROI_SIZE = (150, 150)
INPUT_SIZE = (224, 224)
min_conf = 0.85

In [5]:
# load our network weights from disk
label = 'cirsium setosum'
impath = r'C:\Users\jrsquan\Desktop\DS785\weed-datasets-master\{}.jpg'.format(label)
imbeforepath = r'C:\Users\jrsquan\Desktop\DS785\weed-datasets-master\{} before.jpg'.format(label)
imafterpath = r'C:\Users\jrsquan\Desktop\DS785\weed-datasets-master\{} after.jpg'.format(label)

print("[INFO] loading network...")
model = load_model('weights/ResNet50.hp.weights.hdf5')
#model = load_model('weights/EfficientNetB0.hp.weights.hdf5')

# load the input image from disk, resize it such that it has the
# has the supplied width, and then grab its dimensions
orig = cv2.imread(impath)
orig = imutils.resize(orig, width=WIDTH)
(H, W) = orig.shape[:2]

[INFO] loading network...


In [6]:
# initialize the image pyramid
pyramid = image_pyramid(orig, scale=PYR_SCALE, minSize=ROI_SIZE)
# initialize two lists, one to hold the ROIs generated from the image
# pyramid and sliding window, and another list used to store the
# (x, y)-coordinates of where the ROI was in the original image
rois = []
locs = []

In [7]:
# time how long it takes to loop over the image pyramid layers and
# sliding window locations
start = time.time()

# loop over the image pyramid
for image in pyramid:
    # determine the scale factor between the *original* image
    # dimensions and the *current* layer of the pyramid
    scale = W / float(image.shape[1])
    # for each layer of the image pyramid, loop over the sliding
    # window locations
    for (x, y, roiOrig) in sliding_window(image, WIN_STEP, ROI_SIZE):
        # scale the (x, y)-coordinates of the ROI with respect to the
        # *original* image dimensions
        x = int(x * scale)
        y = int(y * scale)
        w = int(ROI_SIZE[0] * scale)
        h = int(ROI_SIZE[1] * scale)
        # take the ROI and preprocess it so we can later classify
        # the region using Keras/TensorFlow
        roi = cv2.resize(roiOrig, INPUT_SIZE)
        roi = img_to_array(roi)
        roi = preprocess_input(roi)
        # update our list of ROIs and associated coordinates
        rois.append(roi)
        locs.append((x, y, x + w, y + h))
            
# show how long it took to loop over the image pyramid layers and
# sliding window locations
end = time.time()
print("[INFO] looping over pyramid/windows took {:.5f} seconds".format(end - start))
# convert the ROIs to a NumPy array
rois = np.array(rois, dtype="float32")
# classify each of the proposal ROIs using ResNet and then show how
# long the classifications took
print("[INFO] classifying ROIs...")
start = time.time()
preds = model.predict(rois)
end = time.time()
print("[INFO] classifying ROIs took {:.5f} seconds".format(end - start))

[INFO] looping over pyramid/windows took 0.06194 seconds
[INFO] classifying ROIs...
[INFO] classifying ROIs took 6.95786 seconds


In [8]:
pred_classes = np.argmax(preds, axis=1)
counts = np.bincount(pred_classes)
mostFreqClass = np.argmax(counts)
classes = {0: 'bluegrass', 1: 'chenopodium album', 2: 'cirsium setosum', 3: 'corn', 4: 'sedge'}
class_list = [classes[x] for x in pred_classes]
pred_val = np.amax(preds, axis=1)
preds1 = list(zip(class_list, pred_val))
#print(preds1)

In [9]:
# decode the predictions and initialize a dictionary which maps class
# labels (keys) to any ROIs associated with that label (values)
#preds = imagenet_utils.decode_predictions(preds, top=1)

labels = {}
# loop over the predictions
for (i, p) in enumerate(preds1):
    # grab the prediction information for the current ROI
    #(imagenetID, label, prob) = p[0]
    (label, prob) = p
    # filter out weak detections by ensuring the predicted probability
    # is greater than the minimum probability
    if label == classes[mostFreqClass]:
        if prob >= min_conf:
            # grab the bounding box associated with the prediction and
            # convert the coordinates
            box = locs[i]
            # grab the list of predictions for the label and add the
            # bounding box and probability to the list
            L = labels.get(label, [])
            L.append((box, prob))
            labels[label] = L

In [10]:
# loop over the labels for each of detected objects in the image
for label in labels.keys():
    # clone the original image so that we can draw on it
    print("[INFO] showing results for '{}'".format(label))
    clone = orig.copy()
    # loop over all bounding boxes for the current label
    for (box, prob) in labels[label]:
        # draw the bounding box on the image
        (startX, startY, endX, endY) = box
        cv2.rectangle(clone, (startX, startY), (endX, endY),
            (0, 255, 0), 2)
    # show the results *before* applying non-maxima suppression, then
    # clone the image again so we can display the results *after*
    # applying non-maxima suppression
    
    #cv2.imshow("Before", clone)
    cv2.imwrite(imbeforepath, clone)
    clone = orig.copy()
    # extract the bounding boxes and associated prediction
    # probabilities, then apply non-maxima suppression
    boxes = np.array([p[0] for p in labels[label]])
    proba = np.array([p[1] for p in labels[label]])
    boxes = non_max_suppression(boxes, proba)
    # loop over all bounding boxes that were kept after applying
    # non-maxima suppression
    for (startX, startY, endX, endY) in boxes:
        # draw the bounding box and label on the image
        cv2.rectangle(clone, (startX, startY), (endX, endY),
            (0, 255, 0), 2)
        y = startY - 10 if startY - 10 > 10 else startY + 10
        cv2.putText(clone, label, (startX, y),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)
    # show the output after apply non-maxima suppression
    #cv2.imshow("After", clone)
    cv2.imwrite(imafterpath, clone)
    #cv2.waitKey(0)

[INFO] showing results for 'cirsium setosum'
